In [18]:
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러 올 수 있다
import requests

# 리퀘스트로 가져온 웹사이트의 내용과 정보를 불러올 수 있다
from bs4 import BeautifulSoup

#대량 데이터 처리시 진행상황을 알 수 있다
from tqdm import tqdm, trange 
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 20})
# Window 의 한글 폰트 설정
plt.rc('font',family='Malgun Gothic')
%matplotlib inline

In [16]:
! pip install requests

In [19]:
#url sort = p 인기순 

def get_suggestion_list(singers, pages):
    suggestion_list = []
    suggestion_album = []
    singer_list = [] 
    for singer in singers: 
        for page in range(pages): 
            base_url = requests.get("https://music.bugs.co.kr/search/track?q=" + singer+'&flac_only=false&target=ARTIST_TRACK_ALBUM&page='+str(page+1)+'&sort=P')
            if base_url.status_code == 200 : 
                soup = BeautifulSoup(base_url.text, 'html.parser', from_encoding="utf8")
                tags = soup.select('#DEFAULT0 > table > tbody')[0].find_all('tr')
                for tag in tags:
                    suggestion_list.append('https://music.bugs.co.kr/track/'+tag['trackid'])
                    suggestion_album.append('https://music.bugs.co.kr/album/'+tag['albumid'])
                    singer_list.append(singer)
            
    return suggestion_list, suggestion_album, singer_list

In [20]:
get_suggestion_list("Lauv",1)

(['https://music.bugs.co.kr/track/31705498',
  'https://music.bugs.co.kr/track/66966',
  'https://music.bugs.co.kr/track/85854',
  'https://music.bugs.co.kr/track/82633',
  'https://music.bugs.co.kr/track/5598004',
  'https://music.bugs.co.kr/track/33273',
  'https://music.bugs.co.kr/track/27654',
  'https://music.bugs.co.kr/track/33268',
  'https://music.bugs.co.kr/track/33271',
  'https://music.bugs.co.kr/track/30370362',
  'https://music.bugs.co.kr/track/1765251',
  'https://music.bugs.co.kr/track/2960147',
  'https://music.bugs.co.kr/track/5763702',
  'https://music.bugs.co.kr/track/489233',
  'https://music.bugs.co.kr/track/80301392',
  'https://music.bugs.co.kr/track/66964',
  'https://music.bugs.co.kr/track/5463985',
  'https://music.bugs.co.kr/track/80097256',
  'https://music.bugs.co.kr/track/80282596',
  'https://music.bugs.co.kr/track/31068174',
  'https://music.bugs.co.kr/track/5005963',
  'https://music.bugs.co.kr/track/30653655',
  'https://music.bugs.co.kr/track/1702982'

In [21]:
def get_suggestion_content(url_list):
    base_url2 = requests.get(url_list)
    soup2 = BeautifulSoup(base_url2.text, 'html.parser', from_encoding="utf8")
    lyrics_list = [] 
    tags1 = soup2.select('#container > header > div')[0].find_all('h1')
    title = tags1[0].get_text(strip=True).replace('(inst.)','')
    tags2 = soup2.select('#container > section.sectionPadding.contents.lyrics > div > div')[0].find_all('xmp')
    try : 
        lyrics = tags2[0].get_text(strip=True).replace('\r\n',' ').replace('\r',' ').replace('\n',' ')
    except : 
        lyrics = '' 
    #container > section.sectionPadding.summaryInfo.summaryTrack > div > div.basicInfo > table > tbody > tr:nth-child(1) > th
    tags3 = soup2.select('#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.basicInfo > table > tbody')[0].find_all('td')
    artist = tags3[0].get_text(strip=True)
    
    #container > section.sectionPadding.summaryInfo.summaryTrack > div > div.etcInfo > span > a > span > em
    tags4 = soup2.select('#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.etcInfo > span > a > span')[0].find_all('em')
    like = tags4[0].get_text(strip=True).replace(',','')

    
    lyrics_list.append(artist)
    lyrics_list.append(title)
    lyrics_list.append(like)
    lyrics_list.append(lyrics)
        
    return lyrics_list 

In [22]:
def get_suggestion_album(url_list):
    base_url2 = requests.get(url_list)
    soup2 = BeautifulSoup(base_url2.text, 'html.parser', from_encoding="utf8")
    info = []
    tags1 = soup2.select('#container > section.sectionPadding.summaryInfo.summaryAlbum > div > div.basicInfo > table')[0].find_all('time')
    datetime = tags1[0].get_text(strip=True)
    #container > section.sectionPadding.summaryInfo.summaryAlbum > div > div.basicInfo > table > tbody > tr:nth-child(4) > td
    tags2 = soup2.select('#container > section.sectionPadding.summaryInfo.summaryAlbum > div > div.basicInfo > table > tbody')[0].find_all('td')
    genre = tags2[4].get_text(strip=True)
       
    info.append(datetime)
    info.append(genre)
        
    return info

In [ ]:
get_suggestion_album("https://music.bugs.co.kr/track/31705498")

In [26]:
def get_data(singer, pages): 
    data1= []
    data2= [] 
    
    suggestion_list = get_suggestion_list(singer,pages)[0]
    info_list = get_suggestion_list(singer,pages)[1]  
    singer_list = get_suggestion_list(singer,pages)[2]
    
    for i, suggestion in tqdm(enumerate(suggestion_list)):
        s = get_suggestion_content(suggestion)
        data1.append(s)
    
    for i, info in tqdm(enumerate(info_list)):
        s = get_suggestion_album(info)
        data2.append(s)
        
    data1 = pd.DataFrame(data1, columns=["Artist","Title","Like","Lyrics"])
    data2 = pd.DataFrame(data2, columns=["Datetime","Genre"])
    data3 = pd.DataFrame(singer_list, columns=["Keyword"])
    data = pd.concat([data3, data1, data2], axis = 1)
    
    return data 

###  ------------------------------------------- 여기까지 클롤링 불러오는 코드  --------------------------------------------

In [98]:
singer = ["Lauv"]
data = get_data(singer, 1) #페이지1에 곡 50개 

C:\Users\USER\Anaconda3\lib\site-packages\bs4\__init__.py:185: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")

0it [00:00, ?it/s]
1it [00:00,  1.62it/s]
2it [00:00,  1.93it/s]
3it [00:01,  1.86it/s]
4it [00:01,  2.01it/s]
5it [00:02,  2.05it/s]
6it [00:02,  2.06it/s]
7it [00:03,  1.94it/s]
8it [00:04,  1.46it/s]
9it [00:05,  1.53it/s]
10it [00:05,  1.73it/s]
11it [00:05,  1.83it/s]
12it [00:06,  1.96it/s]
13it [00:07,  1.74it/s]
14it [00:07,  1.81it/s]
15it [00:08,  1.92it/s]
16it [00:08,  2.28it/s]
17it [00:08,  2.25it/s]
18it [00:09,  2.14it/s]
19it [00:09,  2.18it/s]
20it [00:10,  2.32it/s]
21it [00:10,  2.14it/s]
22it [00:11,  2.10it/s]
23it [00:11,  2.41it/s]
24it [00:11,  2.57it/s]
25it [00:12,  2.44it/s]
26it [00:12,  2.66it/s]
27it [00:12,  3.09it/s]
28it [00:12,  3.45it/s]
29it

In [31]:
### 가수 'Lauv' 불러오기
data.head(50)

,Keyword,Artist,Title,Like,Lyrics,Datetime,Genre
0,Lauv,Lauv(라우브)CONNECT 아티스트,Paris In The Rain,15036,All I know is ooh ooh ooh We could go anywhere...,2018.10.08,팝
1,Lauv,Lauv(라우브)CONNECT 아티스트,Changes,3413,I'm getting rid of my clothes I don't wear I t...,2020.01.03,팝
2,Lauv,Lauv(라우브)CONNECT 아티스트,"fuck, i'm lonely (feat. Anne-Marie)",5797,I call you one time two time three time I Can...,2019.08.02,팝
3,Lauv,"Lauv(라우브)CONNECT 아티스트,Troye Sivan(트로이 시반)CONNE...",i'm so tired...,10448,I'm so tired of love songs Tired of love songs...,2019.01.25,팝
4,Lauv,방탄소년단,Make It Right (feat. Lauv),3702,I was lost I was tryna find the answer In the ...,2019.10.18,힙합
5,Lauv,Lauv(라우브)CONNECT 아티스트,I Like Me Better,7831,To be young and in love in new york city To no...,2018.02.13,팝
6,Lauv,Lauv(라우브)CONNECT 아티스트,Never Not,4349,We were so beautiful We were so tragic No othe...,2018.10.08,팝
7,Lauv,Lauv(라우브)CONNECT 아티스트,Feelings,2452,Is my love too much? Is it just enough for you...,2019.09.20,팝
8,Lauv,"Lauv(라우브)CONNECT 아티스트,LANY(레이니)",Mean It,1568,Small talk no conversation That look makes me ...,2019.11.15,팝
9,Lauv,Lauv(라우브)CONNECT 아티스트,Who (feat. BTS),1821,Outlines of your eyes and visions of you Girl ...,2020.03.06,팝


In [32]:
data["Lyrics"]

0     All I know is ooh ooh ooh We could go anywhere...
1     I'm getting rid of my clothes I don't wear I t...
2     I call you one time two time three time I  Can...
3     I'm so tired of love songs Tired of love songs...
4     I was lost I was tryna find the answer In the ...
5     To be young and in love in new york city To no...
6     We were so beautiful We were so tragic No othe...
7     Is my love too much? Is it just enough for you...
8     Small talk no conversation That look makes me ...
9     Outlines of your eyes and visions of you Girl ...
10    Another life another story She walked out said...
11    I wish that you and I lived in the Sims We cou...
12    Said I'd never fall, I'd never fall, I'd never...
13    Daydream Life feels like a daydream And I just...
14    Stick stick to your stones cause that's all yo...
15    I've been thinkin' 'bout my father lately The ...
16    I wish that you and I lived in the Sims We cou...
17    To be young and in love in New York City T

### 다른시도.. 토픽 모델링

In [87]:
count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english', ngram_range=(1,2))
feat_vect = count_vect.fit_transform(data["Lyrics"])
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (50, 1000)


In [90]:
### LDA 클래스 
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=5, random_state=0)
lda.fit(feat_vect)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [91]:
def display_topic_words(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('\nTopic #',topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        ### argsort - feature name을 가져오기 위해 위치를 반환해주는 함수 사용
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' + '.join([str(feature_names[i])+'*'+str(round(topic[i],1)) for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vect.get_feature_names()

# Topic별 가장 연관도가 높은 word를 15개만 추출
display_topic_words(lda, feature_names, 15)


Topic # 0
right*75.0 + make*43.0 + oh*42.3 + right right*30.2 + oh make*27.2 + make right*27.2 + knows*20.2 + cause*17.0 + know*17.0 + knows knows*16.2 + lonely*16.1 + right oh*15.2 + set*14.2 + feel*13.8 + head*11.0

Topic # 1
let*67.3 + wouldn*53.2 + wouldn let*46.2 + let wouldn*34.2 + wish*30.5 + easy*26.2 + met*26.2 + love*25.8 + easy love*24.2 + baby*23.8 + night*23.5 + ll*21.3 + getting*20.2 + know*19.0 + way*19.0

Topic # 2
don*67.8 + lonely*63.2 + come*53.7 + lonely lonely*45.2 + know*38.0 + ooh*37.4 + don know*36.0 + hold*27.2 + come hold*27.2 + want*24.9 + don want*24.2 + hit*24.2 + did*21.2 + friends*21.2 + fuck*19.1

Topic # 3
love*153.0 + mmm*96.2 + tired*95.2 + tired love*93.2 + wanna*89.2 + home*78.1 + just*73.6 + songs*67.2 + love songs*67.2 + wanna home*67.2 + songs tired*66.2 + mmm mmm*66.2 + yeah*53.3 + home wanna*44.2 + cause*43.5

Topic # 4
don*156.5 + wanna*93.2 + don wanna*87.2 + sad*59.2 + wanna sad*54.2 + need*47.2 + like*45.2 + know*39.8 + stay*33.5 + better*

In [92]:
doc_topics = lda.transform(feat_vect)
print(doc_topics.shape)
print(doc_topics[:3])

(50, 5)
[[1.38759849e-01 1.87181618e-03 3.07356213e-01 2.46813235e-01
  3.05198887e-01]
 [2.14958723e-03 1.24366104e-01 6.51553007e-01 2.12038499e-03
  2.19810917e-01]
 [5.82345456e-04 5.81630079e-04 9.97672122e-01 5.81847745e-04
  5.82055189e-04]]


In [102]:
import pandas as pd 

topic_names = ['Topic #'+ str(i) for i in range(0, 5)]
doc_topic_df = pd.DataFrame(data=doc_topics, columns=topic_names, index=data["Title"])
doc_topic_df.head(20)

,Topic #0,Topic #1,Topic #2,Topic #3,Topic #4
Title,,,,,
Paris In The Rain,0.138760,0.001872,0.307356,0.246813,0.305199
Changes,0.002150,0.124366,0.651553,0.002120,0.219811
"fuck, i'm lonely (feat. Anne-Marie)",0.000582,0.000582,0.997672,0.000582,0.000582
i'm so tired...,0.000513,0.000513,0.000514,0.997947,0.000514
Make It Right (feat. Lauv),0.998005,0.000499,0.000498,0.000499,0.000499
I Like Me Better,0.001160,0.001169,0.001162,0.001172,0.995337
Never Not,0.003937,0.003950,0.003963,0.918474,0.069675
Feelings,0.000758,0.000760,0.000758,0.996963,0.000761
Mean It,0.000881,0.000888,0.000878,0.775467,0.221886


In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(text_lyrics)
X_tfidf_vect = tfidf_vect.transform(text_lyrics)

In [109]:
def display_topic_words(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('\nTopic #',topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        ### argsort - feature name을 가져오기 위해 위치를 반환해주는 함수 사용
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' + '.join([str(feature_names[i])+'*'+str(round(topic[i],1)) for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = tfidf_vect.get_feature_names()

# Topic별 가장 연관도가 높은 word를 15개만 추출
display_topic_words(lda, feature_names, 15)


Topic # 0
one*75.0 + leave*43.0 + minute*42.3 + other*30.2 + mmm*27.2 + letters*27.2 + hands*20.2 + better*17.0 + got*17.0 + hard*16.2 + internet*16.1 + or*15.2 + promises*14.2 + cried*13.8 + face*11.0

Topic # 1
hesitation*67.3 + together*53.2 + tomorrow*46.2 + hits*34.2 + thorns*30.5 + collar*26.2 + loneliness*26.2 + knew*25.8 + come*24.2 + along*23.8 + memories*23.5 + insane*21.3 + down*20.2 + got*19.0 + their*19.0

Topic # 2
changes*67.8 + internet*63.2 + brings*53.7 + invisible*45.2 + got*38.0 + moonlight*37.4 + chasing*36.0 + fifth*27.2 + broke*27.2 + tastes*24.9 + chilling*24.2 + fault*24.2 + castles*21.2 + do*21.2 + door*19.1

Topic # 3
knew*153.0 + love*96.2 + slip*95.2 + small*93.2 + sure*89.2 + fingers*78.1 + fun*73.6 + remember*67.2 + laughing*67.2 + sweat*67.2 + remembering*66.2 + lucky*66.2 + toy*53.3 + first*44.2 + better*43.5

Topic # 4
changes*156.5 + sure*93.2 + chillin*87.2 + paranoid*59.2 + sweeter*54.2 + matter*47.2 + hours*45.2 + got*39.8 + rocks*33.5 + arms*33.1

In [39]:
text_lyrics = data["Lyrics"]
#text_lyrics = pd.DataFrame(text_lyrics)
#text_lyrics.head(1)

### 문장 토큰화, 단어 토큰화

In [82]:
from nltk import word_tokenize, sent_tokenize

#여러개의 문장으로 된 입력 데이터를 문장별로 단어 토큰화 만드는 함수 생성
def tokenize_text(text):
    
    # 문장별로 분리 토큰
    sentences = sent_tokenize(text)
    # 분리된 문장별 단어 토큰화
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens


#여러 문장들에 대해 문장별 단어 토큰화 수행. 
#word_tokens = tokenize_text(text_lyrics)
#print(type(word_tokens),len(word_tokens))
#print(word_tokens)

In [ ]:
word_tokens = tokenize_text(text_lyrics)

In [43]:
text_sample = 'All I know is ooh ooh ooh We could go anywhere'
word_tokens = tokenize_text(text_sample)
print(word_tokens)

[['All', 'I', 'know', 'is', 'ooh', 'ooh', 'ooh', 'We', 'could', 'go', 'anywhere']]


In [44]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행. 
cnt_vect = CountVectorizer()
cnt_vect.fit(text_lyrics)
X_train_cnt_vect = cnt_vect.transform(text_lyrics)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (50, 989)


### 여기서 가장 많이 나온 단어를 추출하는 방법?....

In [48]:
print(X_train_cnt_vect)

  (0, 1)	2
  (0, 13)	4
  (0, 15)	1
  (0, 22)	1
  (0, 23)	10
  (0, 28)	1
  (0, 30)	9
  (0, 34)	1
  (0, 41)	3
  (0, 46)	2
  (0, 56)	1
  (0, 71)	1
  (0, 85)	2
  (0, 104)	1
  (0, 114)	2
  (0, 122)	4
  (0, 131)	1
  (0, 139)	1
  (0, 147)	4
  (0, 156)	3
  (0, 176)	1
  (0, 183)	1
  (0, 185)	1
  (0, 191)	3
  (0, 195)	4
  :	:
  (49, 658)	1
  (49, 679)	9
  (49, 683)	1
  (49, 689)	2
  (49, 690)	24
  (49, 694)	1
  (49, 709)	1
  (49, 715)	2
  (49, 717)	1
  (49, 729)	1
  (49, 730)	1
  (49, 744)	2
  (49, 745)	1
  (49, 751)	2
  (49, 754)	1
  (49, 771)	32
  (49, 772)	3
  (49, 795)	1
  (49, 807)	1
  (49, 828)	24
  (49, 851)	1
  (49, 863)	1
  (49, 866)	8
  (49, 885)	4
  (49, 889)	3


In [53]:
cnt_vect = CountVectorizer(max_features=1, stop_words='english')


cnt_vect.fit(text_lyrics)
ftr_vect = cnt_vect.transform(text_lyrics)
print(type(ftr_vect), ftr_vect.shape)
print(cnt_vect.vocabulary_)

<class 'scipy.sparse.csr.csr_matrix'> (50, 1)
{'don': 0}


In [54]:
print(ftr_vect)

  (0, 0)	4
  (1, 0)	2
  (2, 0)	16
  (3, 0)	1
  (5, 0)	1
  (8, 0)	37
  (9, 0)	1
  (10, 0)	2
  (13, 0)	47
  (15, 0)	11
  (17, 0)	1
  (20, 0)	3
  (21, 0)	1
  (22, 0)	3
  (23, 0)	6
  (24, 0)	4
  (26, 0)	3
  (28, 0)	4
  (30, 0)	1
  (31, 0)	1
  (33, 0)	2
  (34, 0)	2
  (35, 0)	3
  (40, 0)	16
  (41, 0)	8
  (43, 0)	6
  (44, 0)	47
  (45, 0)	16
  (46, 0)	6
  (47, 0)	12
  (49, 0)	1


In [58]:
pprint(text_lyrics.vocab().most_common(10))

Pretty printing has been turned ON


In [61]:
df = data[data['Artist']=='Lauv(라우브)CONNECT 아티스트']
data[data['Artist']=='Lauv(라우브)CONNECT 아티스트'].head(3)

,Keyword,Artist,Title,Like,Lyrics,Datetime,Genre
0,Lauv,Lauv(라우브)CONNECT 아티스트,Paris In The Rain,15036,All I know is ooh ooh ooh We could go anywhere...,2018.10.08,팝
1,Lauv,Lauv(라우브)CONNECT 아티스트,Changes,3413,I'm getting rid of my clothes I don't wear I t...,2020.01.03,팝
2,Lauv,Lauv(라우브)CONNECT 아티스트,"fuck, i'm lonely (feat. Anne-Marie)",5797,I call you one time two time three time I Can...,2019.08.02,팝


In [63]:
df['Lyrics'].isnull().sum()

0

In [72]:
data = data['Lyrics'].dropna().reset_index()

In [73]:
data.head(5)

,index,Lyrics
0,0,All I know is ooh ooh ooh We could go anywhere...
1,1,I'm getting rid of my clothes I don't wear I t...
2,2,I call you one time two time three time I Can...
3,3,I'm so tired of love songs Tired of love songs...
4,4,I was lost I was tryna find the answer In the ...


In [79]:
sample_content = data['Lyrics'][40]

#sample_content = pd.DataFrame(sample_content)
type(sample_content)

str

In [67]:
def preprocessing(text):
    # 개행문자 제거
    text =  text.strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [81]:
import re

sample_content = preprocessing(sample_content)
sample_content[:1000]

'I call you one time two time three time I Can t wait no more Your Fingers through my hair That s on my mind I know it s been a minute Since you walked right through that door But I still think about you all the time I Don t know I don t know How I m gonna make it out I don t know I don t know Now You got me sayin  Fuck I m lonely I m lonely I m lonely as Fuck come hold me Come hold me come hold me It s been me myself and why Did You go  Did you go  Oh fuck I m lonely I m lonely I m lonely lonely I Yeah I still watch the shows you showed me I still drink that wine But these days it tastes More bitter than sweet hmm And all my friends are way too drunk To save me from my phone So sorry if I say some things I mean I don t know I don t know How I m gonna make it out I don t know I don t know Now you got me sayin  Fuck I m lonely I m lonely I m lonely as Fuck come hold me Come hold me come hold me It s been me myself and why Did you go  Did you go  Oh fuck I m lonely I m lonely I m lonely 

In [83]:
word_tokens = tokenize_text(sample_content)
print(word_tokens)

[['I', 'call', 'you', 'one', 'time', 'two', 'time', 'three', 'time', 'I', 'Can', 't', 'wait', 'no', 'more', 'Your', 'Fingers', 'through', 'my', 'hair', 'That', 's', 'on', 'my', 'mind', 'I', 'know', 'it', 's', 'been', 'a', 'minute', 'Since', 'you', 'walked', 'right', 'through', 'that', 'door', 'But', 'I', 'still', 'think', 'about', 'you', 'all', 'the', 'time', 'I', 'Don', 't', 'know', 'I', 'don', 't', 'know', 'How', 'I', 'm', 'gon', 'na', 'make', 'it', 'out', 'I', 'don', 't', 'know', 'I', 'don', 't', 'know', 'Now', 'You', 'got', 'me', 'sayin', 'Fuck', 'I', 'm', 'lonely', 'I', 'm', 'lonely', 'I', 'm', 'lonely', 'as', 'Fuck', 'come', 'hold', 'me', 'Come', 'hold', 'me', 'come', 'hold', 'me', 'It', 's', 'been', 'me', 'myself', 'and', 'why', 'Did', 'You', 'go', 'Did', 'you', 'go', 'Oh', 'fuck', 'I', 'm', 'lonely', 'I', 'm', 'lonely', 'I', 'm', 'lonely', 'lonely', 'I', 'Yeah', 'I', 'still', 'watch', 'the', 'shows', 'you', 'showed', 'me', 'I', 'still', 'drink', 'that', 'wine', 'But', 'these', 

In [84]:
word_list = pd.Series(word_tokens)
result = word_list.value_counts().head(20)
result

[I, call, you, one, time, two, time, three, time, I, Can, t, wait, no, more, Your, Fingers, through, my, hair, That, s, on, my, mind, I, know, it, s, been, a, minute, Since, you, walked, right, through, that, door, But, I, still, think, about, you, all, the, time, I, Don, t, know, I, don, t, know, How, I, m, gon, na, make, it, out, I, don, t, know, I, don, t, know, Now, You, got, me, sayin, Fuck, I, m, lonely, I, m, lonely, I, m, lonely, as, Fuck, come, hold, me, Come, hold, me, come, hold, me, It, s, ...]    1
dtype: int64

### TF-IDF 기반

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(text_lyrics)
X_tfidf_vect = tfidf_vect.transform(text_lyrics)

In [105]:
X_tfidf_vect

<50x989 sparse matrix of type '<class 'numpy.float64'>'
	with 4516 stored elements in Compressed Sparse Row format>

In [106]:
from scipy import sparse

coo = sparse.coo_matrix(X_tfidf_vect)
coo

<50x989 sparse matrix of type '<class 'numpy.float64'>'
	with 4516 stored elements in COOrdinate format>

In [107]:
topic_word_indexes = X_tfidf_vect.argsort()[::-1]
top_indexes=topic_word_indexes[:10]

AttributeError: argsort not found